In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/SMSSpamCollection.txt'

In [3]:
f=open(data_file,"r")
target=[]
sms=[]

for line in f:
    line = line.strip()

    if line == "":
        continue

    if line[0:4] == 'spam':
        sms.append(line.split('spam')[1].strip())
        target.append('spam')
    
    if line[0:3]=="ham":
        sms.append(line.split('ham')[1].strip())
        target.append("ham")

f.close()
mydata=pd.DataFrame(list(zip(target,sms)),columns=['target','sms'])

In [4]:
mydata

,target,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [5]:
mydata['target'].value_counts()

ham     4827
spam     747
Name: target, dtype: int64

In [6]:
all_sms=" ".join(mydata['sms'])
ham_sms=" ".join(mydata.loc[mydata['target']=="ham","sms"])
spam_sms=" ".join(mydata.loc[mydata['target']=="spam","sms"])
# spam_sms

In [7]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white',
width=4000,height=2000).generate(all_sms)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

KeyboardInterrupt: 

In [ ]:
wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white',
width=4000,height=2000).generate(ham_sms)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white',
width=4000,height=2000).generate(spam_sms)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

### Naive bayes with simple features
Gaussian: It is used in classification and it assumes that features follow a normal distribution.
Multinomial: used with features which are counts.
Bernoulli: when features are binary

In [ ]:
mydata['length']=[len(x) for x in mydata['sms']]

In [ ]:
mydata.head()

In [ ]:
from ggplot import *

In [ ]:
ggplot(mydata,aes(x='length',color='target'))+geom_density()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [ ]:
mydata_train,mydata_test =train_test_split(mydata,test_size=0.2,random_state=2)

x_train=mydata_train.drop(['sms','target'],1)
y_train=mydata_train['target']
x_test=mydata_test.drop(['sms','target'],1)
y_test=mydata_test['target']
clf=MultinomialNB()

clf.fit(x_train,y_train)
predictions=pd.DataFrame(list(zip(y_test,clf.predict(x_test))),columns=['real','predicted'])
pd.crosstab(predictions['real'],predictions['predicted'])

In [ ]:
mydata['word_will']=[int(np.where("will" in x,1,0)) for x in mydata['sms']]
mydata['word_call']=[int(np.where("call" in x,1,0)) for x in mydata['sms']]
mydata['word_FREE']=[int(np.where("FREE" in x,1,0)) for x in mydata['sms']]
mydata['long']=[int(np.where(len(x)>250,1,0)) for x in mydata['sms']]


In [ ]:
mydata_train,mydata_test =train_test_split(mydata,test_size=0.2,random_state=2)
x_train=mydata_train.drop(['sms','target'],1)
y_train=mydata_train['target']
x_test=mydata_test.drop(['sms','target'],1)
y_test=mydata_test['target']
clf=MultinomialNB()
clf.fit(x_train,y_train)
predictions=pd.DataFrame(list(zip(y_test,clf.predict(x_test))),columns=['real','predicted'])
pd.crosstab(predictions['real'],predictions['predicted'])

these doesnt do very well. lets compare this performance with a simple all default decision
tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf=DecisionTreeClassifier()

In [ ]:
clf.fit(x_train,y_train)
predictions=pd.DataFrame(list(zip(y_test,clf.predict(x_test))),columns=['real','predicted'])
pd.crosstab(predictions['real'],predictions['predicted'])

now lets utilise all the words and how they appear in the entire document. And see if Naive
Bayes can beat this

In [ ]:
def split_into_lemmas(message):
    
    message=message.lower()
    words = TextBlob(message).words
    words_sans_stop=[]
    
    for word in words :
        
        if word in stop:
            
            continue
            
        words_sans_stop.append(word)
        
    return [word.lemma for word in words_sans_stop]

tfidf= TfidfVectorizer(analyzer=split_into_lemmas)